# 对象

## == 与 is

`==` 比较两个对象的值 (对象中保存的数据)，而 `is` 比较对象的标识。

`a == b` 等同于 `a.__eq__(b)` 。继承自 object 的 `__eq__` 方法比较两个对象的 ID ，结果与 `is` 一样。
但多数内置类型使用更有意义的方式覆盖了 `__eq__` 方法，会考虑对象属性的值。

# 弱引用

弱引用不会增加对象的引用数量，不会妨碍所指对象被当作垃圾回收。

弱引用是可调用对象，如果对象存在，调用弱引用可以获取对象，否则返回 None 。

弱引用在缓存应用中很有用，因为一般不希望由于被缓存引用着而始终保存缓存对象。

In [1]:
import weakref
a = {0, 1}
wref = weakref.ref(a)
wref()

{0, 1}

`weakref.ref` 类是低层接口，应该多使用 WeakKeyDictionary, WeakValueDictionary, WeakSet 和 finalize ，不要自己动手创建并处理 `weakref.ref` 实例。

### weakref.finalize

In [2]:
import weakref
s = {1, 2, 3}
ender = weakref.finalize(s, lambda: print("Gone with the wind ..."))
ender.alive

True

In [3]:
del s  # del 不会删除对象，但是执行 del 后可能会导致对象不可获取，从而被删除

Gone with the wind ...


### weakref.WeakValueDictionary

WeakValueDictionary 类实现的是一种可变映射，**里面的值是对象的弱引用**。被引用的对象在程序中的其他地方被当作垃圾回收后，对应的键会自动从 WeakValueDictionary 中删除。因此，WeakValueDictionary 经常用于缓存。

In [4]:
import weakref

class Cheese:
    
    def __init__(self, kind):
        self.kind = kind

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Read Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese

sorted(stock.keys())    

['Brie', 'Parmesan', 'Read Leicester', 'Tilsit']

In [5]:
del catalog
del cheese
sorted(stock.keys())

[]

### weakref.WeakKeyDictionary

与 WeakValueDictionary 对应的是 WeakKeyDictionary ，后者的键是弱引用。

### weakref.WeakSet

保存元素弱引用的集合类。元素没有强引用时，集合会把它删除。

如果一个类需要知道所有实例，一种好的方案是创建一个 WeakSet 类型的类属性，用以保存实例的引用。

### 弱引用的局限

不是每个 Python 对象都可以作为弱引用的目标 (或称所指对象) 。list 和 dict 实例不能作为所指对象，但是它们的子类可以。int 和 tuple 实例不能作为弱引用的目标，甚至它们的子类也不行。这些局限是内部优化导致的结果。

## `__slots__`

默认情况下，Python 在各个实例中名为 `__dict__` 的字典里存储实例属性。为了使用底层的散列表提升访问速度，字典会消耗大量内存。如果要处理数百万个属性不多的实例，通过 `__slots__`  类属性，能节省大量内存。其本质是让解释器在元组中存储实例属性，而不是使用字典。

继承自超类的 `__slots__` 属性没有效果，Python 只会使用各个类中定义的 `__slots__` 属性。

不要使用 `__slots__` 禁止类的用户新增实例属性，使用 `__slots__` 是为了优化，不是为了约束。

### 节省的内存也可能被再次吃掉

如果把 `__dict__` 这个名称添加到 `__slots__` 中，实例会在元祖中保存各个实例的属性，同时还支持动态创建属性，但这样就失去了节省内存的功效。

### `__weakref__`

为了让对象支持弱引用，必须要有这个属性。用户定义的类中默认就有这个属性。如果类中定义了 `__slots__` 属性，而且想把实例作为弱引用的目标，那么必须把 `__weakref__` 添加到 `__slots__` 中。